In [3]:
#!/usr/bin/env python3
import re
import os
import subprocess
from datetime import timedelta
from youtube_transcript_api import YouTubeTranscriptApi
from yt_dlp import YoutubeDL
import openai
import srt
from dotenv import load_dotenv
from tqdm import tqdm
import shutil
import sys

# Ensure ffmpeg is available
if not shutil.which("ffmpeg"):
    sys.stderr.write("ERROR: ffmpeg not found on your PATH. Please install ffmpeg.\n")
    sys.exit(1)

# Load OpenAI API key from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def fetch_youtube_transcript(video_id: str):
    """
    Fetch the raw transcript entries for a given YouTube video ID.
    Returns a list of dicts with 'start', 'duration', and 'text'.
    """
    return YouTubeTranscriptApi.get_transcript(video_id)


def translate_text(text: str, target_language: str) -> str:
    """
    Translate a piece of text into the target language via OpenAI (v1 API).
    """
    messages = [
        {"role": "system", "content": f"Translate the following text into {target_language}. Only return the translated text."},
        {"role": "user",   "content": text}
    ]
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return resp.choices[0].message.content.strip()


def write_srt(transcript, translations, filename: str) -> str:
    """
    Write an .srt file given the original transcript and translated lines.
    Each subtitle runs from its start timestamp to the next entry's start timestamp,
    ensuring continuous alignment.
    """
    subs = []
    for i, (entry, text) in enumerate(zip(transcript, translations), start=1):
        start = timedelta(seconds=entry['start'])
        if i < len(transcript):
            end = timedelta(seconds=transcript[i]['start'])
        else:
            end = start + timedelta(seconds=entry.get('duration', 3.0))
        subs.append(srt.Subtitle(index=i, start=start, end=end, content=text))
    srt_content = srt.compose(subs)
    with open(filename, "w", encoding="utf-8") as f:
        f.write(srt_content)
    return filename


def download_video(url: str, filename: str) -> str:
    """
    Download the YouTube video as MP4 via yt-dlp.
    """
    ydl_opts = {"format": "mp4", "outtmpl": filename}
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return filename


def burn_subtitles(video_path: str, srt_path: str, output_path: str) -> str:
    """
    Burn subtitles into the video with dark blue background and white font.
    """
    style = (
        "FontName=Arial,"
        "FontSize=24,"
        "PrimaryColour=&H00FFFFFF&,"
        "BackColour=&H8B0000&,"
        "BorderStyle=3"
    )
    cmd = [
        "ffmpeg", "-y",
        "-i", video_path,
        "-vf", f"subtitles={srt_path}:force_style='{style}'",
        "-c:a", "copy",
        output_path
    ]
    subprocess.run(cmd, check=True)
    return output_path


def process_video(url: str, target_language: str) -> str:
    """
    Full pipeline: download video, fetch + translate transcript, write SRT, burn subtitles.
    Returns the output video filename.
    """
    vid_match = re.search(r"(?:v=|/)([0-9A-Za-z_-]{11})", url)
    if not vid_match:
        raise ValueError("Invalid YouTube URL")
    vid = vid_match.group(1)

    print("Downloading video...", flush=True)
    video_file = download_video(url, f"{vid}.mp4")
    print("Download complete.")

    transcript = fetch_youtube_transcript(vid)

    print("Translating transcript...")
    translations = [
        translate_text(entry['text'], target_language)
        for entry in tqdm(transcript, desc="Translation Progress", unit="chunk")
    ]
    print("Translation complete.")

    srt_file = f"{vid}_{target_language}.srt"
    write_srt(transcript, translations, srt_file)

    output_video = f"{vid}_{target_language}.mp4"
    print("Burning subtitles into video...")
    for _ in tqdm(range(1), desc="Burning Progress", unit="step"):
        burn_subtitles(video_file, srt_file, output_video)
    print("Burn-in complete.")

    return output_video


def parse_user_prompt(prompt: str):
    """
    Extract the YouTube URL and target language from free-form input.
    """
    url_pattern = r"https?://(?:www\.)?youtu(?:\.be/|be\.com/watch\?v=)([0-9A-Za-z_-]{11})"
    url_search = re.search(url_pattern, prompt)
    if not url_search:
        raise ValueError("Could not find a valid YouTube URL in your input.")
    url = url_search.group(0)

    # Look for 'into X' or 'in X'
    lang_search = re.search(r"(?:into|in)\s+([A-Za-z ]+)", prompt, re.IGNORECASE)
    if lang_search:
        language = lang_search.group(1).strip()
    else:
        # fallback prompt
        language = input("Please enter the target language: ")
    return url, language




In [6]:
if __name__ == "__main__":
    print("=== YouTube Subtitle Chat Agent ===")
    print("Send me a sentence like: 'Please subtitle https://www.youtube.com/watch?v=XYZ into Persian'\nType 'exit' to quit.")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ('exit', 'quit', 'bye'):
            print("Goodbye!")
            break
        try:
            url, lang = parse_user_prompt(user_input)
            print(f"Processing video {url} into {lang}...")
            result = process_video(url, lang)
            print(f"✅ Done! Saved subtitled video as {result}\n")
        except Exception as e:
            print(f"Error: {e}\n")

=== YouTube Subtitle Chat Agent ===
Send me a sentence like: 'Please subtitle https://www.youtube.com/watch?v=XYZ into Persian'
Type 'exit' to quit.
Processing video https://www.youtube.com/watch?v=rEDzUT3ymw4 into Persion...
[youtube] Extracting URL: https://www.youtube.com/watch?v=rEDzUT3ymw4
[youtube] rEDzUT3ymw4: Downloading webpage
[youtube] rEDzUT3ymw4: Downloading tv client config
[youtube] rEDzUT3ymw4: Downloading player 9a279502-main
[youtube] rEDzUT3ymw4: Downloading tv player API JSON
[youtube] rEDzUT3ymw4: Downloading ios player API JSON
[youtube] rEDzUT3ymw4: Downloading m3u8 information
[info] rEDzUT3ymw4: Downloading 1 format(s): 18
[download] Destination: rEDzUT3ymw4.mp4
[download] 100% of    1.40MiB in 00:00:01 at 1.31MiB/s   
Download complete.
Translating transcript...


Translation Progress: 100%|██████████| 15/15 [00:16<00:00,  1.12s/chunk]


Translation complete.
Burning subtitles into video...


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Burn-in complete.
✅ Done! Saved subtitled video as rEDzUT3ymw4_Persion.mp4

Goodbye!
